In [3]:
import os

import numpy as np
import pandas as pd
import tqdm
import time

In [4]:
clicks_data = pd.read_csv(
    os.path.join("..", "..", "data", "yoochoose", "yoochoose-clicks.dat"),
    names=['sessionID', 'timestamp', 'itemID', 'category'],
    parse_dates=["timestamp"],
)

buys_data = pd.read_csv(
    os.path.join("..", "..", "data", "yoochoose", "yoochoose-buys.dat"),
    names=['sessionID', 'timestamp', 'itemID', 'price', 'qty'],
    parse_dates=["timestamp"]
)

/tmp/ipykernel_17890/2834650445.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  clicks_data = pd.read_csv(


In [5]:
buys_data.drop(columns=['price', 'qty'], inplace=True)
clicks_data.drop(columns=['category'], inplace=True)

In [6]:
buys_data['type'] = 1
clicks_data['type'] = 0

In [7]:
data = pd.concat((clicks_data, buys_data), axis=0, ignore_index=True)
data = data[["sessionID", "itemID", "timestamp", "type"]]

In [12]:
max_session_type = data.groupby(["sessionID", "itemID"])["type"].max().rename("max_type")

In [13]:
data = data.join(max_session_type, ["sessionID", "itemID"])

In [14]:
data = data[data["type"] == data["max_type"]]

In [15]:
data.reset_index(drop=True, inplace=True)
data.drop(columns="max_type", inplace=True)

In [18]:
click_weight, purchase_weight = \
    data.shape[0] / data['type'].value_counts()

In [19]:
data = data.join(
    data.groupby(['sessionID', 'itemID'])['type'].count().rename('count'),
    on=['sessionID', 'itemID']
)

In [22]:
data.sort_values(by='timestamp', inplace=True)
data.drop_duplicates(['sessionID', 'itemID'], inplace=True)

In [24]:
click_border = click_weight * (purchase_weight // click_weight)

In [25]:
data['weight'] = np.where(
    data['type'] == 0,
    np.minimum(click_border, data['count'] * click_weight),
    data['count'] * purchase_weight
)

In [27]:
data.drop(columns=['type', 'count'], inplace=True)

In [29]:
data.timestamp = data.timestamp.astype(int)

In [31]:
data.to_parquet(os.path.join('..', '..', 'data', 'yoochoose', 'interactions.parquet'), index=False)